In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import os
import dotenv

This step improts the FIPS data and subsets it to the group of desired states in our anlaysis. 

In [113]:
# Importing FIPS
url = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county"


response = requests.get(url)
html_content = response.content



soup = BeautifulSoup(html_content, "html.parser")



table = soup.find("table", {"class": "wikitable"})



rows = table.find_all("tr")


data = []


for row in rows[1:]:  # Skip the header row
    cols = row.find_all("td")


    if len(cols) >= 2:  # Ensure the row has at least two columns


        fips_code = cols[0].text.strip()


        county_name = cols[1].text.strip()


        data.append({"FIPS Code": fips_code, "County Name": county_name})



county_fips_df = pd.DataFrame(data)



county_fips_df.to_csv("county_fips.csv", index=False)



print(county_fips_df.head(10))

  FIPS Code      County Name
0     01001   Autauga County
1     01003   Baldwin County
2     01005   Barbour County
3     01007      Bibb County
4     01009    Blount County
5     01011   Bullock County
6     01013    Butler County
7     01015   Calhoun County
8     01017  Chambers County
9     01019  Cherokee County


Below we add a mapping for:
- Each State FIPS (first 2 digits of a FIPS code)
- The corresponsing 2 letter abbreviation for these states

In [114]:
# Adjusting FIPS
first_2 = ["53", "01", "13", "23", "08", "12", "40", "41"]


mapping = {
    "53": "WA",
    "01": "AL",
    "13": "GA",
    "23": "ME",
    "08": "CO",
    "12": "FL",
    "40": "OK",
    "41": "OR",
}



county_fips_df["state_FIPS"] = county_fips_df["FIPS Code"].str[:2]



fips_df = county_fips_df[county_fips_df["state_FIPS"].isin(first_2)].copy()



fips_df["State"] = fips_df["state_FIPS"].map(mapping)



fips_df = fips_df.rename(columns={"FIPS Code": "County_FIPS", "County Name": "County"})



print(len(fips_df))

525


In [115]:
# IMporting Educaiton, Poverty, Unemployment
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
url = "https://raw.githubusercontent.com/MIDS-at-Duke/opioids-2024-data-queens-king/main/data/USDA_education_poverty_unemployment_income.parquet"
headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
response = requests.get(url, headers=headers)
epu_df = pd.read_parquet("USDA_education_poverty_unemployment_income.parquet")
epu_df["FIPS"] = epu_df["FIPS"].apply(lambda x: str(x).zfill(5))
epu_df["YR"].value_counts()
desired_years = ["2008-2012", "2015", "2014", "2017", "2016", "2021"]
epu_df1 = epu_df[epu_df["YR"].isin(desired_years)]
epu_df1.sample(5)

,FIPS,Name,RUC Code,YR,Completing College,Completing High School Only,Not Completing High School,All people in poverty (2021) Percent,Children ages 0-17 in poverty (2021) Percent,Median Household Income (2021),Unemployment Rate (%)
5915,40031,None,NaN,2021,None,None,None,19.2,23.4,"$50,090",4.3
7364,41069,None,NaN,2016,None,None,None,NaN,NaN,None,4.1
1536,08065,None,NaN,2015,None,None,None,NaN,NaN,None,3.7
6446,40103,"Noble, OK",8.0,2008-2012,19.3%,35.9%,12.8%,NaN,NaN,None,NaN
1414,08049,"Grand, CO",9.0,2008-2012,32.3%,27.8%,6.0%,NaN,NaN,None,NaN


In [116]:
# This code expands the '2008-2012' lines into individual lines for 2008, 2009, 2010, 2011. 2012
expanded_rows = []
for index, row in epu_df1.iterrows():
    if "-" in row["YR"]:
        start, end = map(int, row["YR"].split("-"))
        for year in range(start, end + 1):
            new_row = row.copy()
            new_row["YR"] = year
            expanded_rows.append(new_row)
    else:
        expanded_rows.append(row)
epu_df2 = pd.DataFrame(expanded_rows)
epu_df2.reset_index(drop=True, inplace=True)
print(epu_df2.head(30))

     FIPS                 Name  RUC Code    YR Completing College  \
0   01000              Alabama       NaN  2008              22.3%   
1   01000              Alabama       NaN  2009              22.3%   
2   01000              Alabama       NaN  2010              22.3%   
3   01000              Alabama       NaN  2011              22.3%   
4   01000              Alabama       NaN  2012              22.3%   
5   01000                 None       NaN  2014               None   
6   01000                 None       NaN  2015               None   
7   01000                 None       NaN  2016               None   
8   01000                 None       NaN  2017               None   
9   01000                 None       NaN  2021               None   
10  01001  Autauga, AL               2.0  2008              21.7%   
11  01001  Autauga, AL               2.0  2009              21.7%   
12  01001  Autauga, AL               2.0  2010              21.7%   
13  01001  Autauga, AL            

In [117]:
# Importing population
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
url_2 = "https://raw.githubusercontent.com/MIDS-at-Duke/opioids-2024-data-queens-king/main/data/USDA_population2021.parquet"
headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
response = requests.get(url_2, headers=headers)
with open("USDA_population2021.parquet", "wb") as file:
    file.write(response.content)
pop = pd.read_parquet("USDA_population2021.parquet")

# Include preceding 0 on FIPS
pop["FIPS"] = pop["FIPS"].apply(lambda x: str(x).zfill(5))
pop = pop.rename(columns={"FIPS": "County_FIPS"})
pop.head(5)

,County_FIPS,State,Name,YR,Estimated Population
0,01000,AL,Alabama,2021,"5,050,380"
1,01001,AL,Autauga County,2021,"59,203"
2,01003,AL,Baldwin County,2021,"239,439"
3,01005,AL,Barbour County,2021,"24,533"
4,01007,AL,Bibb County,2021,"22,359"


In [118]:
# Merging FIPS on pop

fips_pop = pd.merge(
    fips_df, pop, how="left", on=["County_FIPS"], indicator=True, validate="1:1"
)

missing_values = fips_pop.isna().sum()

# print(f"By computing the total missing values in fips_pop after the merge the following is obtained:{missing_values}")


fips_pop.head(5)
fips_pop = fips_pop[
    [
        "County_FIPS",
        "County",
        "state_FIPS",
        "State_x",
        "YR",
        "Estimated Population",
        "_merge",
    ]
]
fips_pop = fips_pop.rename(
    columns={
        "State_x": "State",
        "YR": "pop_Year",
        "Estimated Population": "pop_Population",
        "_merge": "pop_merge_indicator",
    }
)

fips_pop["pop_merge_indicator"].value_counts()
# print(f"By computing the total missing values in fips_pop after the merge the following is obtained:{missing_values}")

pop_merge_indicator
both          525
left_only       0
right_only      0
Name: count, dtype: int64

In [119]:
# Importing Poverty Data
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
url_3 = "https://raw.githubusercontent.com/MIDS-at-Duke/opioids-2024-data-queens-king/main/data/USDA_poverty2021.parquet"
headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
response = requests.get(url_3, headers=headers)
with open("USDA_poverty2021.parquet", "wb") as file:
    file.write(response.content)
pov = pd.read_parquet("USDA_poverty2021.parquet")

# Include preceding 0 on FIPS
pov["FIPS"] = pov["FIPS"].apply(lambda x: str(x).zfill(5))
pov = pov.rename(columns={"FIPS": "County_FIPS"})
pov = pov[["County_FIPS", "All people in poverty (2021) Percent"]]
pov = pov.rename(
    columns={"All people in poverty (2021) Percent": "pov_poverty_rate_percent"}
)
pov.head(5)

,County_FIPS,pov_poverty_rate_percent
0,23000,11.2
1,23001,14.0
2,23003,14.9
3,23005,7.7
4,23007,11.7


In [120]:
# Merging FIPS_POP on POV
fpp = pd.merge(
    fips_pop, pov, how="left", on=["County_FIPS"], indicator=True, validate="1:1"
)
fpp = fpp.rename(columns={"_merge": "pov_merge_indicator"})

fpp["pov_merge_indicator"].value_counts()

pov_merge_indicator
both          525
left_only       0
right_only      0
Name: count, dtype: int64

In [121]:
fpp["County"] = fpp["County"].str.upper().str.replace(" COUNTY", "", regex=False)
fpp.head(20)

,County_FIPS,County,state_FIPS,State,pop_Year,pop_Population,pop_merge_indicator,pov_poverty_rate_percent,pov_merge_indicator
0,01001,AUTAUGA,01,AL,2021,"59,203",both,10.7,both
1,01003,BALDWIN,01,AL,2021,"239,439",both,10.8,both
2,01005,BARBOUR,01,AL,2021,"24,533",both,23.0,both
3,01007,BIBB,01,AL,2021,"22,359",both,20.6,both
4,01009,BLOUNT,01,AL,2021,"59,079",both,12.0,both
5,01011,BULLOCK,01,AL,2021,"10,143",both,32.1,both
6,01013,BUTLER,01,AL,2021,"18,890",both,22.7,both
7,01015,CALHOUN,01,AL,2021,"115,678",both,19.2,both
8,01017,CHAMBERS,01,AL,2021,"34,488",both,19.7,both
9,01019,CHEROKEE,01,AL,2021,"25,074",both,18.2,both


In [122]:
# Importing Opioid Shipment Data
file_path = "data/opioid_shipment_WA_FL_andconstants.parquet"


opioid = pd.read_parquet(file_path)

county_adj_dict_op = {"DE KALB": "DEKALB"}


opioid["BUYER_COUNTY"].replace(county_adj_dict_op)

opioid.loc[
    (opioid["BUYER_COUNTY"].str.startswith("DE", na=False))
    & (opioid["BUYER_STATE"] == "AL"),
    "BUYER_COUNTY",
] = opioid["BUYER_COUNTY"].str.replace(" ", "", regex=False)



opioid["state-county"] = opioid["BUYER_STATE"] + "|" + opioid["BUYER_COUNTY"]


opioid.head(2)


opioid["state-county-yr"] = opioid["state-county"] + "|" + opioid["year"].astype("str")


unique_id_of_op = opioid["state-county-yr"].nunique()


length_op = len(opioid)


print(
    f"the length of the opioid dataset is {unique_id_of_op} while the distinctcount of the unique id state-county-yr is {length_op}"
)

opioid["year"].value_counts()

the length of the opioid dataset is 7128 while the distinctcount of the unique id state-county-yr is 7128


year
2006    513
2011    511
2010    511
2014    511
2009    510
2007    510
2013    510
2012    510
2008    509
2016    508
2019    507
2018    507
2015    506
2017    505
Name: count, dtype: int64

In [123]:
opioid = opioid.rename(
    columns={
        "year": "opioid_YEAR",
        "BUYER_STATE": "opioid_State",
        "BUYER_COUNTY": "opioid_County",
        "morphine_equivalent_g": "opioid_morphine_equivalent_g",
        "state-county": "state_county",
        "state-county-yr": "opioid_state_county_yr",
    }
)

opioid_unique_counties = opioid[
    ["state_county", "opioid_State", "opioid_County"]
].drop_duplicates()

In [124]:
county_adj_dict = {
    "DE KALB": "DEKALB",
    "ST. CLAIR": "SAINT CLAIR",
    "BROOMFIELD, CITY AND OF[I]": "BROOMFIELD",
    "DENVER, CITY AND OF[J]": "DENVER",
    "DESOTO": "DE SOTO",
    "ST. JOHNS": "SAINT JOHNS",
    "ST. LUCIE": "SAINT LUCIE",
}
fpp["County"] = fpp["County"].replace(county_adj_dict)


fpp["state_county"] = fpp["State"] + "|" + fpp["County"]
fpp_opioid = pd.merge(
    fpp, opioid, how="left", on=["state_county"], indicator=True, validate="1:m"
)

fpp_opioid = fpp_opioid.rename(columns={"_merge": "opioid_merge_indicator"})

fpp_opioid["opioid_merge_indicator"].value_counts()

left_only = fpp_opioid[fpp_opioid["opioid_merge_indicator"] == "left_only"]

print(left_only)

     County_FIPS         County state_FIPS State pop_Year pop_Population  \
1106       08023       COSTILLA         08    CO     2021          3,613   
1158       08033        DOLORES         08    CO     2021          2,375   
1307       08057        JACKSON         08    CO     2021          1,359   
3014       13053  CHATTAHOOCHEE         13    GA     2021          9,044   
3336       13101         ECHOLS         13    GA     2021          3,698   
4264       13239        QUITMAN         13    GA     2021          2,242   
4421       13265     TALIAFERRO         13    GA     2021          1,569   
6483       41055        SHERMAN         41    OR     2021          1,907   

     pop_merge_indicator  pov_poverty_rate_percent pov_merge_indicator  \
1106                both                      23.1                both   
1158                both                      13.9                both   
1307                both                      14.8                both   
3014               

Below is a skeleton model to merge the main opioid data with the FIPS Dataset. I assume here that the naming convention for each county is consistent between the FIPS reference table and the main Opioid Dataset. If not, an additional dictionary wil be needed where names get fixed prior to the merge. This Merge Connects: 
- The Main Opioid Dataset
- FIPS Code for each county

In [125]:
df3 = pd.DataFrame(
    {
        "FIPS": [134, 234, 333],
        "County": ["County A", "County B", "County C"],

        "State": ["AB", "BB", "AA"],
    }
)



df3["state-county"] = df3["State"] + "|" + df3["County"]



df1 = pd.DataFrame(
    {
        "Year": [2005, 2006, 2007],
        "State": ["AB", "BB", "AA"],
        "County": ["County A", "County B", "County C"],

        "Opioid": [10, 20, 30],
    }
)



df1["state-county"] = df1["State"] + "|" + df1["County"]


df1 = pd.merge(
    df1, df3, how="left", on=["state-county"], indicator=True, validate="m:1"
)



print(df1.head(3))

   Year State_x  County_x  Opioid state-county  FIPS  County_y State_y _merge
0  2005      AB  County A      10  AB|County A   134  County A      AB   both
1  2006      BB  County B      20  BB|County B   234  County B      BB   both
2  2007      AA  County C      30  AA|County C   333  County C      AA   both


Below is some dummy data to test my merge operation prior to setting it in main_merge_viz. In this instance we merge two datasets:
- The Main Opioid Dataset
- The Dataset that contains Population per FIPS (or population per FIPS-year)

In [126]:
# df1 = pd.DataFrame({
#     'FIPS': [134, 234, 333],
#     'Year': [2005,2006,2007],
#     'State': ['GA', 'FL', 'WA'],
#     'County': ['County A', 'County B', 'County C']
#     'Value1': [10, 20, 30]
# })


# df2 = pd.DataFrame({
#     'FIPS':[134, 234, 333],
#     'County Population':[1000, 200, 3000],
# })

In [127]:
# Testing Merge Functions assuring we get a One-One OR Many-One Merge

In [128]:
# merged_data = pd.merge(df1,df2, how='left', on=['FIPS'], indicator=True, validate='m:1')

# print(merged_data.sample(2))